# ML API Specification for EO Data Cubes
### Brian Pondi and Rolf Simoes

This notebook demonstrates the ML API processes for the openEO ecosystem, structured to address four pillars: initialization of model architectures, execution of training/prediction tasks, and management of model artifacts. By decoupling ML logic from backend implementations, the API enables portable workflows that are reusable across infrastructures.

![ML API Architecture](./eo-ml.png)

## Design Principles

The core principles that guided the API development include:

- **Modularity**
- **Consistency**
- **Backend-Agnosticism** 
- **Default Parameters**
- **Extensible**

## Model Initialization

The model initialization phase, facilitated by processes prefixed with `mlm_`, is designed to create untrained model definition objects. Each process name follows the pattern:

```
mlm_<type>_<model>
```

where `<type>` is an abbreviated ML action category (e.g., `class` for classification, `regr` for regression, `segm` for segmentation, `gen` for generative, etc.) and `<model>` specifies the particular algorithm (e.g., `random_forest`, `svm`, `xgboost`, `tempcnn`, `tae`, etc.).

| Process Name              | Description                                                               |
| ------------------------- | ------------------------------------------------------------------------- |
| `mlm_class_catboost`      | Initializes a CatBoost classification model                               |
| `mlm_class_mlp`           | Initializes a Multi-Layer Perceptron (MLP) classification model           |
| `mlm_class_random_forest` | Initializes a Random Forest classification model                          |
| `mlm_class_svm`           | Initializes a Support Vector Machine (SVM) classification model           |
| `mlm_class_xgboost`       | Initializes an XGBoost classification model                               |
| `mlm_class_tempcnn`       | Initializes a Temporal Convolutional Neural Network (TempCNN) model       |
| `mlm_class_tae`           | Initializes a Temporal Attention Encoder (TAE) model                      |
| `mlm_class_lighttae`      | Initializes a lightweight version of the Temporal Attention Encoder model |
| `mlm_regr_svm`            | Initializes a Support Vector Machine regression model                     |
| `mlm_regr_random_forest`  | Initializes a Random Forest regression model                              |

## Model Actions

Prefixed with `ml_`, model action processes are responsible for executing key ML workflows on model artifacts and EO data cubes. These actions include model training, prediction, uncertainty estimation, and post-processing.

| Process Name               | Description                                                                                |
| -------------------------- | ------------------------------------------------------------------------------------------ |
| `ml_fit`                   | Fits a machine learning model to a data cube of input features and target values           |
| `ml_predict`               | Applies a trained machine learning model to a data cube of input features                  |
| `ml_predict_probabilities` | Applies a model to input features and returns predicted class probabilities                |
| `ml_uncertainty_class`     | Estimates classification uncertainty using methods like margin, ratio, or least-confidence |
| `ml_smooth_class`          | Applies spatial smoothing to classification probability results using Bayesian inference   |
| `ml_label_class`           | Converts a probability data cube to a labeled data cube                                    |

## Model Management

Model management processes handle the storage and retrieval of ML artifacts. They enable users to export and import preliminary or private models into their workspace and to export and import models using STAC ML Model extension compliance.

| Process Name      | Description                                                                              |
| ----------------- | ---------------------------------------------------------------------------------------- |
| `import_ml_model` | Imports a previously exported machine learning model from a specified workspace location |
| `export_ml_model` | Exports a machine learning model to a specified workspace location                       |
| `load_ml_model`   | Loads a machine learning model from a STAC:MLM Item into the current session             |
| `save_ml_model`   | Saves a machine learning model with STAC MLM Extension compliance                        |

# Temporal CNN Example

This section demonstrates how to train a Temporal CNN (TempCNN) model using OpenEO processes. The example uses deforestation data from Rondonia to train a deep learning model for time series classification.

## Setup and Connection

First, we load the required libraries and connect to the OpenEO backend.

In [1]:
import openeo # type: ignore
from rpy2 import robjects
from rpy2.robjects.packages import importr
jsonlite = importr('jsonlite')

/Users/brianpondi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Error importing in API mode: ImportError("dlopen(/Users/brianpondi/Library/Python/3.9/lib/python/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <EF10DC06-064E-3614-96B8-350E8279C9FD> /Users/brianpondi/Library/Python/3.9/lib/python/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRbla

In [2]:
connection = openeo.connect(
    url="http://127.0.0.1:8000",
    auth_type="basic",
    auth_options={"username": "brian", "password": "123456"}
)

In [3]:
print(connection.list_collections())

[{'type': 'Collection', 'stac_version': '1.0.0', 'id': 'mpc-landsat-c2-l2', 'title': 'Landsat Collection 2 Level-2', 'description': 'Landsat Collection 2 Level-2 [Science Products](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products), consisting of atmospherically corrected [surface reflectance](https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-reflectance) and [surface temperature](https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-temperature) image data. Collection 2 Level-2 Science Products are available from August 22, 1982 to present.\n\nThis dataset represents the global archive of Level-2 data from [Landsat Collection 2](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2) acquired by the [Thematic Mapper](https://landsat.gsfc.nasa.gov/thematic-mapper/) onboard Landsat 4 and 5, the [Enhanced Thematic Mapper](https://landsat.gsfc.nasa.gov/the-enhanced-thematic-mapper-plus-etm/) onboard Landsa

In [4]:
print(connection.list_collection_ids())

['mpc-landsat-c2-l2', 'mpc-sentinel-2-l2a', 'mpc-sentinel-1-grd', 'mpc-sentinel-1-rtc']


## Explore Available Processes

Let's examine the available processes on the backend, particularly focusing on ML-related ones.


In [5]:
process_ids = [process["id"] for process in connection.list_processes()]
print("Available processes on this backend:")
for process_id in process_ids:
    print(f"- {process_id}")

Available processes on this backend:
- load_collection
- mlm_class_random_forest
- mlm_class_svm
- mlm_class_xgboost
- ml_class_mlp
- mlm_class_tempcnn
- mlm_class_tae
- mlm_class_lighttae
- ml_fit
- ml_predict
- ml_predict_probabilities
- ml_uncertainty_class
- ml_smooth_class
- ml_label_class
- cube_regularize
- ndvi
- save_result
- load_result
- export_cube
- import_cube
- export_ml_model
- import_ml_model
- save_ml_model
- load_ml_model


## Examine ML Process Details

Let's look at the details of some ML processes to understand their parameters and requirements.


In [6]:
connection.describe_process("mlm_class_random_forest")

{'id': 'mlm_class_random_forest',
 'summary': 'Initialize a Random Forest classification model',
 'description': 'Creates and configures a Random Forest classification model. Parameters such as the number of trees and maximum depth can be specified.',
 'categories': ['machine learning'],
 'experimental': True,
 'parameters': [{'name': 'num_trees',
   'description': 'The number of trees in the Random Forest.',
   'schema': {'type': 'integer', 'minimum': 1, 'default': 100}},
  {'name': 'max_variables',
   'description': 'Specifies how many split variables will be used at a node.\n\nThe following options are available:\n\n- *integer*: The given number of variables are considered for each split.\n- `log2`: The logarithm with base 2 of the number of variables are considered for each split.\n- `sqrt`: The square root of the number of variables are considered for each split. This is often the default for classification.',
   'optional': True,
   'default': 'sqrt',
   'schema': [{'type': 'integer', 'minimum': 1},
    {'type': 'string', 'enum': ['log2', 'sqrt']}]},
  {'name': 'seed',
   'description': 'A randomization seed to use for the random sampling in training. If not given or `null`, no seed is used and results may differ on subsequent use.',
   'optional': True,
   'default': {},
   'schema': {'type': ['integer', 'null']}}],
 'returns': {'description': 'An untrained Random Forest classification model instance.',
  'schema': {'type': 'object', 'subtype': 'ml-model'}},
 'links': [{'href': 'https://doi.org/10.1023/A:1010933404324',
   'title': 'Breiman (2001): Random Forests',
   'type': 'text/html',
   'rel': 'about'}]}

In [7]:
connection.describe_process("mlm_class_tempcnn")

{'id': 'mlm_class_tempcnn',
 'summary': 'Initialize a Temporal Convolutional Neural Network (TempCNN) classification model',
 'description': 'Creates and configures a Temporal Convolutional Neural Network (TempCNN) model for time-series data. Parameters such as the convolutional architecture, dense layers, optimizer, and learning rate schedule can be specified.',
 'categories': ['machine learning'],
 'experimental': True,
 'parameters': [{'name': 'cnn_layers',
   'description': 'A list specifying the number of filters in each convolutional layer. Defaults to [64, 64, 64].',
   'optional': True,
   'default': [64, 64, 64],
   'schema': {'type': 'array',
    'items': {'type': 'integer', 'minimum': 1},
    'minItems': 1}},
  {'name': 'cnn_kernels',
   'description': 'A list specifying the kernel size for each convolutional layer. Must match the number of CNN layers. Defaults to [5, 5, 5].',
   'optional': True,
   'default': [5, 5, 5],
   'schema': {'type': 'array',
    'items': {'type': 'integer', 'minimum': 1},
    'minItems': 1}},
  {'name': 'cnn_dropout_rates',
   'description': 'A list of dropout rates for each convolutional layer. Must match the number of CNN layers. Defaults to [0.2, 0.2, 0.2].',
   'optional': True,
   'default': [0.2, 0.2, 0.2],
   'schema': {'type': 'array',
    'items': {'type': 'number', 'minimum': 0, 'maximum': 1},
    'minItems': 1}},
  {'name': 'dense_layer_nodes',
   'description': 'The number of nodes in the dense layer following the convolutional layers. Defaults to 256.',
   'optional': True,
   'default': 256,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'dense_layer_dropout_rate',
   'description': 'The dropout rate for the dense layer. Defaults to 0.5.',
   'optional': True,
   'default': 0.5,
   'schema': {'type': 'number', 'minimum': 0, 'maximum': 1}},
  {'name': 'optimizer',
   'description': "The optimizer to use for training. Defaults to 'adam'. Supported values include 'adam', 'adabound', 'adabelief', 'madagrad', 'nadam', 'qhadam', 'radam', 'swats', and 'yogi'.",
   'optional': True,
   'default': 'adam',
   'schema': {'type': 'string',
    'enum': ['adam',
     'adabound',
     'adabelief',
     'madagrad',
     'nadam',
     'qhadam',
     'radam',
     'swats',
     'yogi']}},
  {'name': 'learning_rate',
   'description': 'The initial learning rate for training. Defaults to 0.0005.',
   'optional': True,
   'default': 0.0005,
   'schema': {'type': 'number', 'minimum': 0}},
  {'name': 'epsilon',
   'description': 'The epsilon value for numerical stability in optimizers. Defaults to 1e-8.',
   'optional': True,
   'default': 1e-08,
   'schema': {'type': 'number', 'minimum': 0}},
  {'name': 'weight_decay',
   'description': 'The weight decay (L2 penalty) value for regularization. Defaults to 1e-6.',
   'optional': True,
   'default': 1e-06,
   'schema': {'type': 'number', 'minimum': 0}},
  {'name': 'lr_decay_epochs',
   'description': 'The number of epochs after which the learning rate is decayed. Defaults to 1.',
   'optional': True,
   'default': 1,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'lr_decay_rate',
   'description': 'The rate at which the learning rate is decayed after the specified number of epochs. Defaults to 0.95.',
   'optional': True,
   'default': 0.95,
   'schema': {'type': 'number', 'minimum': 0, 'maximum': 1}},
  {'name': 'epochs',
   'description': 'The number of training epochs. Defaults to 150.',
   'optional': True,
   'default': 150,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'batch_size',
   'description': 'The size of batches for training. Defaults to 64.',
   'optional': True,
   'default': 64,
   'schema': {'type': 'integer', 'minimum': 1}},
  {'name': 'seed',
   'description': 'Seed for the random number generator. If `null`, no seed is used, and results may vary.',
   'optional': True,
   'default': {},
   'schema': {'type': ['integer', 'null']}}],
 'returns': {'description': 'An untrained TempCNN classif

## Load and Prepare Data

We'll load Sentinel-2 data and prepare it for our analysis.

In [8]:
# Load a data cube
datacube = connection.load_collection(
    collection_id="mpc-sentinel-2-l2a",
    spatial_extent={"west": -63.9, "south": -9.14, "east": -62.9, "north": -8.14},
    temporal_extent=["2022-01-01", "2022-12-31"]
)

Failed to parse API error response: [500] '{"code":500,"message":"Error: $ operator is invalid for atomic vectors"}' (headers: {'Date': 'Sat, 21 Jun 2025 07:22:48 GMT', 'Access-Control-Allow-Origin': '*', 'Content-Type': 'application/json', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'})


OpenEoApiPlainError: [500] {"code":500,"message":"Error: $ operator is invalid for atomic vectors"}

In [ ]:
datacube = datacube.process(
    process_id="cube_regularize",
    arguments={
        "data": datacube,
        "period": "P1M",  # Monthly regularization
        "resolution": 320
    }
)

## Load Training Data

We'll load the pre-processed training data for deforestation in Rondonia.


In [ ]:
# Load the RDS file using rpy2
readRDS = robjects.r['readRDS']
data_deforestation_rondonia = readRDS("./monthly_rondonia_data.rds")

In [ ]:
data_deforestation_rondonia

In [ ]:
# Serialize the data using jsonlite::serializeJSON
serializeJSON = robjects.r['serializeJSON']
# Use the function
serialized_data = serializeJSON(data_deforestation_rondonia)

In [ ]:
serialized_data

## Initialize and Train the Model

Now we'll initialize the Temporal CNN model and train it with our data.

In [ ]:
tempcnn_model_init = connection.mlm_class_tempcnn(
    optimizer="adam",
    learning_rate=0.0005,
    seed=42
)

In [ ]:
# Fit the model using the training dataset
tempcnn_model = tempcnn_model_init.fit(
    training_set=serialized_data,
    target="label"
)

## Make Predictions

Apply the trained model to make predictions on new data.


In [ ]:
datacube =  tempcnn_model.predict(
    data=datacube,
    model=tempcnn_model
)

## Save the Model

Save the trained model for future use.

In [ ]:
tempcnn_model.save_ml_model(name ="tempcnn_rondonia",
                            tasks=["classification"]
                            #options={}
                            )

## Save and Execute Results

Finally, we'll save the prediction results and execute the job.


In [ ]:
result = datacube.save_result(
    format="GTiff"
)

In [ ]:
job = result.create_job(
    title="Deforestation Prediction in Rondonia",
    description="Using TempCNN model to predict deforestation in Rondonia"
)
job

In [ ]:
job.start_and_wait()
job.get_results().download_files("output")

## Conclusion

This example demonstrated how to:
1. Connect to an OpenEO backend
2. Load and prepare training data
3. Define a Temporal CNN model architecture
4. Train the model
5. Make predictions on new data
6. Save the results

The trained model can now be used for making predictions on new time series data. 